# Setup a Phoenix observability dashboard on Hugging Face Spaces for LLM application tracing

[Phoenix](https://docs.arize.com/phoenix) is an open-source observability library by [Arize AI](https://arize.com/) designed for experimentation, evaluation, and troubleshooting. It allows AI Engineers and Data Scientists to quickly visualize their data, evaluate performance, track down issues, and export data to improve.

In the example, we'll see how to deploy a Phoenix observability dashboard on [Hugging Face Spaces](https://huggingface.co/spaces) and configure it to automatically trace LLM calls, providing a comprehensive view into the inner workings of your LLM applications.

## Step 1: Deploy Phoenix on Hugging Face Spaces

While Phoenix offers a [notebook-first option](https://docs.arize.com/phoenix/deployment/environments#notebooks) for local development, it can also be deployed as a [standalone dashboard via Docker](https://docs.arize.com/phoenix/deployment/environments#container). A long-running, hosted dashboard is a great way to provide a centralized view into your LLM application behavior, and to collaborate with your team. Hugging Face Spaces offers a simple way to host ML applications with optional, persistent storage, and it's support for custom Docker images makes it a great platform for hosting Phoenix - lets see how it works!

First, we'll [duplicate the demo space](https://huggingface.co/spaces/andrewrreed/phoenix-arize-observability-demo)

![](./images/duplicate.png)

We can configure the space to be private or public, and it can live in our user namespace, or in an organization namespace. We can use the default, free-tier CPU, and importantly, specify that we want to attach a persistent disk to the space.

> Note: In order for the tracing data to persist across Space, we _must_ configure a persistent disk, otherwise all data will be lost when the space is restarted. Configuring a persistent disk is a paid feature, and will incur a cost for the lifetime of the Space. In this case, we'll use the Small - 20GB disk option for $0.01 per hour.

After clicking "Duplicate Space", the Docker image will begin building. This will take a few minutes to complete, and then we'll see an empty Phoenix dashboard.

![](./images/empty-dashboard.png)


## Step 2: Configure application tracing

Now that we have a running Phoenix dashboard, we can configure our application to automatically trace LLM calls using an [OpenTelemetry TracerProvider](https://docs.arize.com/phoenix/quickstart#connect-your-app-to-phoenix). In this example, we'll instrument our application using the OpenAI client library, and trace LLM calls made from the `openai` Python package to open LLMs running on [Hugging Face's Serverless Inference API](https://huggingface.co/docs/api-inference/en/index).

> Note: Phoenix supports tracing for [a wide variety of LLM frameworks](https://docs.arize.com/phoenix/tracing/integrations-tracing), including LangChain, LlamaIndex, AWS Bedrock, and more.


First, we need to install the necessary libraries:


In [ ]:
!pip install -q arize-phoenix arize-phoenix-otel openinference-instrumentation-openai openai huggingface-hub

Then, we'll login to Hugging Face using the `huggingface_hub` library. This will allow us to generate the necessary authentication for our Space and the Serverless Inference API. Be sure that the HF token used to authenticate has the correct permissions for the Organization where your Space is located.

In [ ]:
from huggingface_hub import interpreter_login

interpreter_login()

Now, we can [configure the Phoenix client](https://docs.arize.com/phoenix/deployment/configuration#client-configuration) to our running Phoenix dashboard:

1. Register the Phoenix tracer provider by
    - Specifying the `project_name` of our choice
    - Setting the `endpoint` value to the Hostname of our Space (found via the dashboard UI under the "Settings" tab - see below)
    - Setting the `headers` to the Hugging Face headers needed to access the Space
2. Instrument our application code to use the OpenAI tracer provider

![](./images/settings.png)

In [1]:
from phoenix.otel import register
from huggingface_hub.utils import build_hf_headers
from openinference.instrumentation.openai import OpenAIInstrumentor

# 1. Register the Phoenix tracer provider
tracer_provider = register(
    project_name="test",
    endpoint="https://andrewrreed-phoenix-arize-observability-demo.hf.space"
    + "/v1/traces",
    headers=build_hf_headers(),
)

# 2. Instrument our application code to use the OpenAI tracer provider
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: test
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://andrewrreed-phoenix-arize-observability-demo.hf.space/v1/traces
|  Transport: HTTP
|  Transport Headers: {'user-agent': '****', 'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Step 3: Make calls and view traces in the Phoenix dashboard

Now, we can make a call to an LLM and view the traces in the Phoenix dashboard. We're using the OpenAI client to make calls to the Hugging Face Serverless Inference API, which is instrumented to work with Phoenix. In this case, we're using the `meta-llama/Llama-3.1-8B-Instruct` model.

In [2]:
from openai import OpenAI
from huggingface_hub import get_token

client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1/",
    api_key=get_token(),
)

messages = [{"role": "user", "content": "What does a llama do for fun?"}]

response = client.chat.completions.create(
    model="meta-llama/Llama-3.1-8B-Instruct",
    messages=messages,
    max_tokens=500,
)

print(response.choices[0].message.content)

While we can't directly ask a llama about their personal preferences, we can make some educated guesses based on their natural behavior.

Llamas are social animals and live in herds in the wild, so they enjoy interacting with each other. Here are some activities that llamas might do for fun:

1. **Play-fighting**: Llamas have a natural inclination to play-fight, which helps them develop social skills and establish dominance hierarchies within their herd. They'll often engage in mock battles, using their heads and necks to spar with each other.

2. **Exploring their surroundings**: Llamas are naturally curious creatures and love to explore their environment. They'll often wander around their enclosure, investigating new sights, smells, and sounds.

3. **Grazing**: Llamas are herbivores, and they love to graze on grasses, leaves, and other plant material. They'll spend hours roaming around, searching for the tastiest and most nutritious food sources.

4. **High-up vantage points**: Llama

If we navigate back to the Phoenix dashboard, we can see the trace from our LLM call is captured and displayed! If you configured your space with a persistent disk, all of the trace information will be saved anytime you restart the space.

![](./images/test-trace.png)



## Bonus: Tracing a multi-agent application with CrewAI

The real power of observability comes from being able to trace and inspect complex LLM workflows. In this example, we'll use [CrewAI](https://www.crewai.com/) to trace a multi-agent application.

In [29]:
!pip install -q openinference-instrumentation-crewai crewai crewai-tools openinference-instrumentation-litellm


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Like before, we'll register the Phoenix tracer provider and instrument the application code, but this time we'll also uninstrument the existing OpenAI tracer provider to avoid conflicts.

In [3]:
from opentelemetry import trace
from openinference.instrumentation.crewai import CrewAIInstrumentor

# 0. Uninstrument existing tracer provider and clear the global tracer provider
OpenAIInstrumentor().uninstrument()
if trace.get_tracer_provider():
    trace.get_tracer_provider().shutdown()
    trace._TRACER_PROVIDER = None  # Reset the global tracer provider

# 1. Register the Phoenix tracer provider
tracer_provider = register(
    project_name="crewai",
    endpoint="https://andrewrreed-phoenix-arize-observability-demo.hf.space"
    + "/v1/traces",
    headers=build_hf_headers(),
)

# 2. Instrument our application code to use the OpenAI tracer provider
CrewAIInstrumentor().instrument(tracer_provider=tracer_provider)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: crewai
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://andrewrreed-phoenix-arize-observability-demo.hf.space/v1/traces
|  Transport: HTTP
|  Transport Headers: {'user-agent': '****', 'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



Now we'll define a multi-agent application using CrewAI to write a blog post about the importance of observability and tracing in LLM applications.

> Note: This example is borrowed and slightly modified from [here](https://docs.arize.com/phoenix/tracing/integrations-tracing/crewai) to change the subject and remove the need for Serper API key.

In [4]:
from huggingface_hub import get_token
from crewai import LLM, Agent, Task, Crew, Process

# Define our LLM using HF's Serverless Inference API
llm = LLM(
    model="huggingface/meta-llama/Llama-3.1-8B-Instruct",
    api_key=get_token(),
    max_tokens=1024,
)

# Define your agents with roles and goals
researcher = Agent(
    role="Senior Research Analyst",
    goal="Uncover cutting-edge developments in AI and data science",
    backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=1,
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    max_iter=1,
)

# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of the importance of observability and tracing in LLM applications.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the importance of observability and tracing in LLM applications.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
    expected_output="Blog post of at least 3 paragraphs",
    agent=writer,
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=True,
    process=Process.sequential,
)

# Get your crew to work!
result = crew.kickoff()

print("------------ FINAL RESULT ------------")
print(result)

2024-10-30 17:33:31,893 - 8143739712 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the importance of observability and tracing in LLM applications.
  Identify key trends, breakthrough technologies, and potential industry impacts.


2024-10-30 17:34:04,017 - 8143739712 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Senior Research Analyst
## Final Answer: 
**Comprehensive Analysis Report: Observability and Tracing in LLM Applications**

**Executive Summary**

Observability and tracing are crucial components in Large Language Model (LLM) applications, enabling developers to monitor, debug, and optimize their complex systems. As LLMs continue to advance, the importance of observability and tracing will only grow. This report highlights key trends, breakthrough technologies, and potential industry impacts of observability and tracing in LLM applications.

**Key Trends:**

* **Increased Adoption of Cloud-Native Observability**: Cloud-native observability platforms are gaining traction, providing real-time monitoring and tracing capabilities for LLM applications.
* **Growing Importance of AI-Powered Observability**: AI-powered observability tools are emerging, leveraging machine learning algorithms to analyze vast amounts of data and provide actionable insights.
* **Rise of Open-Source Obse

2024-10-30 17:35:17,789 - 8143739712 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Tech Content Strategist
## Final Answer: 
**The Importance of Observability and Tracing in LLM Applications: Unlocking Transparency, Trust, and Efficiency**

Observability and tracing are crucial components in Large Language Model (LLM) applications, enabling developers to monitor, debug, and optimize their complex systems. As LLMs continue to advance, the importance of observability and tracing will only grow. This report highlights key trends, breakthrough technologies, and potential industry impacts of observability and tracing in LLM applications.

**Key Trends:**

*   **Increased Adoption of Cloud-Native Observability**: Cloud-native observability platforms are gaining traction, providing real-time monitoring and tracing capabilities for LLM applications.
*   **Growing Importance of AI-Powered Observability**: AI-powered observability tools are emerging, leveraging machine learning algorithms to analyze vast amounts of data and provide actionable insights.
*   **Rise of

After navigating back to the Phoenix dashboard, we can see the traces from our multi-agent application in the new project "crewai"!

![](./images/crewai-trace.png)
